In [3]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.6.12-py3-none-any.whl size=102975 sha256=2bd19568083edd8c5c53826de1695f5f617b39d2f07c45ff652a3ee9a688329d
  Stored in directory: /Users/christie/Library/Caches/pip/wheels/f3/eb/e9/819c2d9eac90204eec8579430759f75a1d6dbe4cd0b93f53bc
Successfully built kaggle


In [13]:
!kaggle datasets download -d yelp-dataset/yelp-dataset

Dataset URL: https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset
License(s): other
100%|██████████████████████████████████████▊| 4.06G/4.07G [00:25<00:00, 175MB/s]
100%|███████████████████████████████████████| 4.07G/4.07G [00:25<00:00, 169MB/s]


In [14]:
!unzip yelp-dataset.zip

Archive:  yelp-dataset.zip
  inflating: Dataset_User_Agreement.pdf  
  inflating: yelp_academic_dataset_business.json  
  inflating: yelp_academic_dataset_checkin.json  
  inflating: yelp_academic_dataset_review.json  
  inflating: yelp_academic_dataset_tip.json  
  inflating: yelp_academic_dataset_user.json  


In [23]:
import os
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Yelp Data Analysis") \
    .getOrCreate()

In [35]:
review_df = spark.read.json('yelp_academic_dataset_review.json')
business_df = spark.read.json('yelp_academic_dataset_business.json')

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=29705Kb max_used=29705Kb free=101366Kb
 bounds [0x000000010a1e8000, 0x000000010bf18000, 0x00000001121e8000]
 total_blobs=11707 nmethods=10772 adapters=846
 compilation: disabled (not enough contiguous free space left)


24/05/07 02:00:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [26]:
review_df.printSchema()
review_df.show(8)

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPp

In [53]:
from pyspark.sql.functions import col, count, when, length

missingdata_review = review_df.select([count(when(col(c).isNull(), c)).alias(c) for c in review_df.columns])
missingdata_review.show()

+-----------+----+----+-----+---------+-----+----+------+-------+
|business_id|cool|date|funny|review_id|stars|text|useful|user_id|
+-----------+----+----+-----+---------+-----+----+------+-------+
|          0|   0|   0|    0|        0|    0|   0|     0|      0|
+-----------+----+----+-----+---------+-----+----+------+-------+



In [54]:
review_df = review_df.withColumn('review_length', length('text'))
review_df.describe('review_length').show()

+-------+-----------------+
|summary|    review_length|
+-------+-----------------+
|  count|          6990280|
|   mean|567.7644364746477|
| stddev|527.2578085249028|
|    min|                1|
|    max|             5000|
+-------+-----------------+



In [43]:
business_df.printSchema()
business_df.show(8)


root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [44]:
missingdata_business = business_df.select([count(when(col(c).isNull(), c)).alias(c) for c in business_df.columns]).show()
missingdata_business 

+-------+----------+-----------+----------+----+-----+-------+--------+---------+----+-----------+------------+-----+-----+
|address|attributes|business_id|categories|city|hours|is_open|latitude|longitude|name|postal_code|review_count|stars|state|
+-------+----------+-----------+----------+----+-----+-------+--------+---------+----+-----------+------------+-----+-----+
|      0|     13744|          0|       103|   0|23223|      0|       0|        0|   0|          0|           0|    0|    0|
+-------+----------+-----------+----------+----+-----+-------+--------+---------+----+-----------+------------+-----+-----+



In [45]:
business_df.groupBy('stars').count().orderBy('stars').show()

+-----+-----+
|stars|count|
+-----+-----+
|  1.0| 1986|
|  1.5| 4932|
|  2.0| 9527|
|  2.5|14316|
|  3.0|18453|
|  3.5|26519|
|  4.0|31125|
|  4.5|27181|
|  5.0|16307|
+-----+-----+



In [50]:
from pyspark.sql.functions import explode, split
business_df.withColumn('category', explode(split('categories', ', '))).groupBy('category').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|            category|count|
+--------------------+-----+
|         Restaurants|52268|
|                Food|27781|
|            Shopping|24395|
|       Home Services|14356|
|       Beauty & Spas|14292|
|           Nightlife|12281|
|    Health & Medical|11890|
|      Local Services|11198|
|                Bars|11065|
|          Automotive|10773|
|Event Planning & ...| 9895|
|          Sandwiches| 8366|
|American (Traditi...| 8139|
|         Active Life| 7687|
|               Pizza| 7093|
|        Coffee & Tea| 6703|
|           Fast Food| 6472|
|  Breakfast & Brunch| 6239|
|      American (New)| 6097|
|     Hotels & Travel| 5857|
+--------------------+-----+
only showing top 20 rows



In [51]:
business_df.groupBy('city').count().orderBy('city').show()

+--------------------+-----+
|                city|count|
+--------------------+-----+
|         AB Edmonton|    1|
|              AMBLER|    1|
|             ARDMORE|    1|
|                AVON|    1|
|            Abington|  169|
|   Abington Township|    3|
|              Affton|   36|
|               Afton|    1|
|Alberta Park Indu...|    1|
|               Aldan|   11|
|             Algiers|    1|
|         Aliso Viejo|    1|
|             Alloway|    2|
|          Almonesson|    1|
|               Alton|  150|
|              Ambler|  211|
|           Andalusia|    3|
|             Antioch|  231|
|        Apollo Beach|  122|
|        Apollo beach|    1|
+--------------------+-----+
only showing top 20 rows

